## Setup

### Imports

In [124]:
import collections
import math
import random
import statistics
import time

from IPython.display import clear_output
import numpy as np

### Helpers

In [125]:
def random_solution(problem):
    solution = problem.copy()
    for block in range(9):
        indices = block_indices(block)
        block = problem[indices]
        zeros = [i for i in indices if problem[i] == 0]
        to_fill = [i for i in range(1, 10) if i not in block]
        random.shuffle(to_fill)
        for index, value in zip(zeros, to_fill):
            solution[index] = value
    return solution

In [126]:
def print_sudoku(sudoku):
    border = "------+-------+------"
    # Find duplicate values
    row_dups = [set() for i in range(9)]
    col_dups = [set() for i in range(9)]
    for i in range(9):
        c = collections.Counter(sudoku[9*i + j] for j in range(9))
        row_dups[i] = [k for k, v in c.items() if v > 1]
    for j in range(9):
        c = collections.Counter(sudoku[9*i + j] for i in range(9))
        col_dups[j] = [k for k, v in c.items() if v > 1]
    # Print sudoku
    for i in range(9):
        if i % 3 == 0:
            print(border)
        for j in range(9):
            if j and j % 3 == 0:
                print("| ", end="")
            v = sudoku[9*i + j]
            mistake = v in row_dups[i] + col_dups[j]  
            if fixed[9 * i + j]:
                print("\x1b[32m", end="")
            elif mistake:
                print("\x1b[31m", end="")
            print(v or " ", end=" ")
            if mistake or fixed[9 * i + j]:
                print("\x1b[0m", end="")
        print()
    print(border)

In [127]:
def get_index(row, col):
    """Convert row/colum numbers to index"""
    return row*9+col

In [128]:
def block_indices(block_num):
    """Return list of indices contained in a block"""
    r, c = map(lambda x: x * 3, divmod(block_num, 3))
    return [get_index(r+i, c+j) for i in range(3) for j in range(3)]

### Create Problem

In [129]:
_ = 0
problem = np.array([
    1, _, _,  _, _, 6,  3, _, 8,
    _, _, 2,  3, _, _,  _, 9, _,
    _, _, _,  _, _, _,  7, 1, 6,

    7, _, 8,  9, 4, _,  _, _, 2,
    _, _, 4,  _, _, _,  9, _, _,
    9, _, _,  _, 2, 5,  1, _, 4,

    6, 2, 9,  _, _, _,  _, _, _,
    _, 4, _,  _, _, 7,  6, _, _,
    5, _, 7,  6, _, _,  _, _, 3,
])

In [130]:
# Create binary mask showing which values are fixed
fixed = problem != 0

In [131]:
print_sudoku(problem)

------+-------+------
1     |     6 | 3   8 
    2 | 3     |   9   
      |       | 7 1 6 
------+-------+------
7   8 | 9 4   |     2 
    4 |       | 9     
9     |   2 5 | 1   4 
------+-------+------
6 2 9 |       |       
  4   |     7 | 6     
5   7 | 6     |     3 
------+-------+------


## Process Definition

### Initialise State

In [132]:
costs = []

In [133]:
# Randomly fill with remaining values from each 3x3 square
solution = problem.copy()
for block in range(9):
    indices = block_indices(block)
    block = problem[indices]
    zeros = [i for i in indices if problem[i] == 0]
    to_fill = [i for i in range(1, 10) if i not in block]
    random.shuffle(to_fill)
    for index, value in zip(zeros, to_fill):
        solution[index] = value

In [134]:
print_sudoku(solution)

------+-------+------
1 3 6 | 5 1 6 | 3 2 8 
9 4 2 | 3 4 2 | 5 9 4 
7 5 8 | 8 9 7 | 7 1 6 
------+-------+------
7 1 8 | 9 4 7 | 5 6 2 
6 3 4 | 6 8 3 | 9 8 7 
9 5 2 | 1 2 5 | 1 3 4 
------+-------+------
6 2 9 | 9 8 3 | 4 8 9 
1 4 8 | 5 1 7 | 6 7 5 
5 3 7 | 6 2 4 | 2 1 3 
------+-------+------


### Setup Cost Function

In [135]:
def cost(sudoku):
    cost = 0
    for i in range(9):
        cost += 9 - len(set(sudoku[get_index(i, j)] for j in range(9)))
        cost += 9 - len(set(sudoku[get_index(j, i)] for j in range(9)))
    return cost

### Process Parameters

In [136]:
# Number of iterations per temperature
iterations = sum(fixed)
# Initial temperature
temp = statistics.pstdev(cost(random_solution(problem))
                         for n in range(1000))
# Temperature decrease factor
df = 1 - 10e-5
# Number of iterations without an improvement before raising temperature
patience = 2000

## Simulate

In [137]:
# Setup
random.seed(253)
found_solution = False
curr_cost = cost(solution)
no_improvement_count = 0
it = 0

while not found_solution:
    
    # Randomly switch two cells
    proposal = solution.copy()
    target = random.choice(range(9))
    indices = block_indices(target)
    non_fixed = [index for index in indices if not fixed[index]]
    i1, i2 = random.choices(non_fixed, k=2)
    proposal[i1], proposal[i2] = proposal[i2], proposal[i1]
    prop_cost = cost(proposal)

    # Adjust temperature
    temp *= df
    if prop_cost >= curr_cost:
        no_improvement_count += 1
    else:
        no_improvement_count = 0
    if no_improvement_count > patience:
        temp += 2
    
    # Probability of accepting
    p = math.exp(-(prop_cost - curr_cost) / temp)
    if random.random() < p or prop_cost == 0:
        solution = proposal.copy()
        curr_cost = prop_cost
    
    # Check for solution
    if curr_cost == 0:
        found_solution = True
    
    # Print sudoku every 1000 iterations
    if it % 1000 == 0:
        clear_output(wait=True)
        print(f"Temp: {temp:.02f} | Cost: {curr_cost}")
        print_sudoku(solution)
    it += 1
clear_output(wait=True)
print_sudoku(solution)
print("\n", "~~~ SOLVED! ~~~".center(20))

------+-------+------
1 7 5 | 4 9 6 | 3 2 8 
8 6 2 | 3 7 1 | 4 9 5 
4 9 3 | 8 5 2 | 7 1 6 
------+-------+------
7 1 8 | 9 4 3 | 5 6 2 
2 5 4 | 1 6 8 | 9 3 7 
9 3 6 | 7 2 5 | 1 8 4 
------+-------+------
6 2 9 | 5 3 4 | 8 7 1 
3 4 1 | 2 8 7 | 6 5 9 
5 8 7 | 6 1 9 | 2 4 3 
------+-------+------

   ~~~ SOLVED! ~~~   
